In [1]:
# Get the path to the desktop directory on macOS
import os
user_directory = os.path.join(os.path.expanduser('~'), '')
os.chdir(user_directory)

In [2]:
import matplotlib.pyplot as plt
import numpy as np

x = np.load("./Downloads/PIPNet/x_pred_LIST.npy")
y = np.load("./Downloads/PIPNet/y_pred_LIST.npy")

x_ref = np.load("./Downloads/PIPNet/X_REF_LIST.npy")
y_ref = np.load("./Downloads/PIPNet/Y_REF_LIST.npy")

x_new = np.zeros((x.shape[0], x.shape[1]))
y_new = np.zeros((y.shape[0], y.shape[1]))

# plt.scatter(x[0,48:],-y[0,48:])
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        x_new[i, j] = x[i, j] + x_ref[i]
        y_new[i, j] = y[i, j] + y_ref[i]

xy_new = np.stack((x_new, y_new))

# Ref to vertical line (nose) point
for i in range(xy_new.shape[1]):
    xy_new[0,i,:] = xy_new[0,i,:] - xy_new[0,i,27]
    xy_new[1,i,:] = -(xy_new[1,i,:] - xy_new[1,i,27])
    
for i in range(x.shape[0]):
    x[i, :] = x[i, :]-x[i, 27]
    y[i, :] = y[i, :]-y[i, 27]

In [3]:
rot_xy_arr = np.zeros((2, xy_new.shape[1], 68))
ang = np.zeros((xy_new.shape[1]))
for i in range(xy_new.shape[1]):
    ang[i]=np.arctan(x[i,62]/y[i,62])
    
    rot=np.array([[np.cos(ang[i]),np.sin(ang[i])], [-np.sin(ang[i]),np.cos(ang[i])]])
    rot_xy = np.dot(rot, np.vstack((y[i, :], x[i, :])))
    rot_xy_arr[1, i, :] = rot_xy[1,:]
    rot_xy_arr[0, i, :] = rot_xy[0,:]
    
rot_xy_arr[0, :, :] = -rot_xy_arr[0, :, :]

In [20]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import librosa
import numpy as np
import os

# ... rest of your code ...

def plot_landmark_motion(landmarks, audio_data, landmark_indices, audio_sample_rate=44100, video_fps=30, output_dir='plots'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    frame_times = np.arange(landmarks.shape[1]) / video_fps
    audio_times = np.arange(len(audio_data)) / audio_sample_rate

    # Determine global min and max for each landmark
    y_landmark_mins = [np.min([landmarks[0][frame][idx] for frame in range(landmarks.shape[1])]) for idx in landmark_indices]
    y_landmark_maxs = [np.max([landmarks[0][frame][idx] for frame in range(landmarks.shape[1])]) for idx in landmark_indices]
    x_landmark_mins = [np.min([landmarks[1][frame][idx] for frame in range(landmarks.shape[1])]) for idx in landmark_indices]
    x_landmark_maxs = [np.max([landmarks[1][frame][idx] for frame in range(landmarks.shape[1])]) for idx in landmark_indices]
    audio_min = np.min(audio_data)
    audio_max = np.max(audio_data)

    total_frames = landmarks.shape[1]

    # Mapping of landmark indices to labels
    landmark_labels = {51: 'Upper Lip', 57: 'Lower Lip', 48: 'Left', 54: 'Right'}

    for i in range(total_frames):
        fig = plt.figure(figsize=(10, 10))  # Adjusted figure size
        spec = gridspec.GridSpec(len(landmark_indices) + 1, 2, height_ratios=[1]*len(landmark_indices) + [2])

        # ... rest of your code ...
        for j, idx in enumerate(landmark_indices):
            x_coords = [landmarks[1][frame][idx] for frame in range(i+1)]
            y_coords = [landmarks[0][frame][idx] for frame in range(i+1)]
            # Get the label for the current landmark index, or use the index if no label is defined
            label = landmark_labels.get(idx, f'Landmark {idx}')
            ax_y = fig.add_subplot(spec[j, 0])
            ax_y.plot(frame_times[:i+1], y_coords, label=f'Landmark {idx} Y', linestyle='--')
            ax_y.set_ylim(y_landmark_mins[j]-2, y_landmark_maxs[j]+2)
            ax_y.set_xlim([0,len(audio_data) / audio_sample_rate])
            ax_y.set_ylabel('Y Position')
            # axs[j, 0].legend(loc='upper right')
            # axs[j, 0].set_title(f'Landmark {idx} Motion (Frame {i})')
            ax_y.set_title(f'{label} Y Motion (Frame {i})')

            # Plot X-coordinates on the right column
            ax_x = fig.add_subplot(spec[j, 1])
            ax_x.plot(frame_times[:i+1], x_coords, label=f'Landmark {idx} X', color='red')
            ax_x.set_ylim(x_landmark_mins[j]-2, x_landmark_maxs[j]+2)
            ax_x.set_xlim([0,len(audio_data) / audio_sample_rate])
            ax_x.set_ylabel('X Position')
            # axs[j, 1].legend(loc='upper right')
            # axs[j, 1].set_title(f'Landmark {idx} X Motion (Frame {i})')
            ax_x.set_title(f'{label} X Motion (Frame {i})')

        # Create a new subplot for the audio signal
        ax_audio = fig.add_subplot(spec[len(landmark_indices), 0])
        current_audio_time = frame_times[i]
        audio_samples_up_to_current_time = audio_data[audio_times <= current_audio_time]
        audio_times_up_to_current_time = audio_times[audio_times <= current_audio_time]
        ax_audio.plot(audio_times_up_to_current_time, audio_samples_up_to_current_time)
        ax_audio.set_ylim(audio_min, audio_max)
        ax_audio.set_xlim([0,len(audio_data) / audio_sample_rate])
        ax_audio.set_xlabel('Time (s)')
        ax_audio.set_ylabel('Amplitude')
        ax_audio.set_title('Audio Signal')

        plt.tight_layout()
        plt.subplots_adjust(hspace=0.4, wspace=0.4)  # Adjusted spacing between subplots
        plt.savefig(f'{output_dir}/plot_{i:04d}.jpg', dpi=200)
        plt.close(fig)
# Select landmarks to plot (e.g., landmarks 0 and 1)
landmark_indices = [51, 57, 48, 54]

# Load the raw audio file
audio_file = './Desktop/Folders/.wav'  # Replace with your audio file path
audio, sample_rate = librosa.load(audio_file, sr=44100)
audio1 = audio[int(60.207 * 44100) : int(60.547 * 44100)]
rot_xy_arr1 = rot_xy_arr[:, int(60.207 * 30) : int(60.547 * 30), :]
# Call the function with your data
plot_landmark_motion(landmarks=rot_xy_arr1, audio_data=audio1, landmark_indices=landmark_indices, audio_sample_rate=44100, video_fps=30, output_dir='./Downloads/plots/')
